<a href="https://colab.research.google.com/github/luasm17/LLM_as_a_judge/blob/main/selene_few_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Instalar dependencias + Imports
!pip install -q -U "transformers>=4.51.0" accelerate safetensors

import os
import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.3/10.3 MB 72.3 MB/s eta 0:00:00


In [2]:
# --- BLOQUE: Prompt (Selene, formato binario) ---
USER_PROMPT = r'''Es un LLM-as-a-judge que vai avaliar un modelo de corrección gramatical (GEC) en galego. En concreto, a túa tarefa é a de avaliar a saída dun corrector gramatical e decidir se o modelo GEC corrixiu correctamente o erro de concordancia de número da frase inicial (é dicir, singular/plural entre determinante, substantivo, adxectivo ou verbo). Lembra que debes cinguirte exclusivamente á corrección ou non da concordancia de número.



Recibirás sempre:

input_corrector: a frase orixinal que se lle pasou ao corrector

output_corrector: a frase devolta polo corrector



Tes que devolver a túa avaliación como LLM-as-a-judge seguindo exactamente este formato; debes respostar exclusivamente a estes catro puntos variables, sen engadir texto adicional:

input_corrector: "<oración de entrada do corrector>"

output_corrector: "<oración xa avaliada polo corrector>"

tag: <0 ou 1>

explanation: "<explicación breve e precisa en galego do motivo polo que se escolleu a etiqueta 0 ou a etiqueta 1>"



Criterios que debes seguir:

tag = 0: a saída do corrector é correcta con respecto á concordancia de número (é dicir, non hai erro de concordancia de número no output_corrector). Nota sobre a tag = 0 (dúas situacións posibles):

Caso 0.A: input_corrector = output_corrector. O input_corrector xa era correcto respecto á concordancia de número e o modelo GEC non fixo cambios adicionais porque non había nada que corrixir. A explicación debe indicar que non había erro de concordancia de número no input.

Caso 0.B: input_corrector ≠ output_corrector. O input_corrector tiña un erro de concordancia de número e o modelo GEC corrixiuno. A explicación debe indicar que erro(s) concordancia se arranxou/arranxaron.

En ambos casos, a etiqueta é 0, pero a explicación debe reflectir se houbo corrección do input_corrector (caso 0.B) ou non por ser xa correcto (caso 0.A).



tag = 1: a saída do corrector non é correcta con respecto á concordancia de número (é dicir, segue habendo erro de concordancia de número no output_corrector). Nota sobre a tag = 1 (dúas situacións posibles):

Caso 1.A: input_corrector = output_corrector. O input_corrector era incorrecto respecto á concordancia de número e o modelo GEC non correxiu o erro presente. A explicación debe indicar que o modelo GEC non corrixiu o erro de concordancia de número presente no input_corrector.

Caso 1.B: input_corrector ≠ output_corrector. O input_corrector non tiña ningún erro de concordancia de número, pero o modelo GEC introduciuno. A explicación debe indicar que o corrector introduciu no output_corrector un erro que non estaba presente no input_corrector.

En ambos casos, a etiqueta é 1, pero a explicación debe reflectir se houbo modificación do input_corrector (caso 1.B) ou non (caso 1.A).



Restricións que debes respectar:

Non debes, baixo ningún concepto, corrixir ou modificar de ningunha forma o input nin o output do corrector.

Tes que limitarte exclusivamente a decidir se o erro de concordancia de número foi corrixido polo modelo GEC ou non.

É imprescindible que te fixes só na concordancia de número, non debes avaliar, baixo ningún concepto, outros tipos de erros.

Non uses nunca o español, o portugués nin nigunha lingua que non sexa o galego nas túas respostas.



Aquí tes uns exemplos de inputs que poderías atopar e dos output que deberías devolver, respectivamente:



Exemplo 1 (caso 0.A): tag = 0

Exemplo de entrada (o que che vai chegar):

input_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

output_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

Exemplo de saída (o que ti tes que devolver):

input_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

output_corrector: "Acaba de saír o sol despois de moita choiva e os nenos corren cara o parque."

tag: 0

explanation: "O output_corrector é adecuado con respecto á concordancia de número xa que non se modificou o input_corrector, o cal non contiña ningún erro de concordancia que corrixir."



Exemplo 2 (caso 0.B): tag = 0

Exemplo de entrada (o que che vai chegar):

input_corrector: "A decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

output_corrector: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "A decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

output_corrector: "As decisións tomadas polo comité foron comunicadas aos responsables das distintas áreas."

tag: 0

explanation: "O output_corrector é adecuado con respecto á concordancia de número porque solucionou o erro do input_corrector entre “A"e “decisións”."



Exemplo 3 (caso 1.A): tag = 1

Exemplo de entrada (o que che vai chegar):

input_corrector: "Pásame ese libro que están enriba da mesa, por favor."

output_corrector: "Pásame ese libro que están enriba da mesa, por favor."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "Pásame ese libro que están enriba da mesa, por favor."

output_corrector: "Pásame ese libro que están enriba da mesa, por favor."

tag: 1

explanation: "O output_corrector non é adecuado con respecto á concordancia de número porque non se modificou o input_corrector, o cal contén un erro de concordancia entre “están"e “libro”."



Exemplo 4 (caso 1.B): tag = 1

Exemplo de entrada (o que che vai chegar):

input_corrector: "Todo o mundo chegou a tempo á xuntanza aquel día."

output_corrector: "Todo o mundo chegaron a tempo á xuntanza aquel día."



Exemplo de saída (o que ti tes que devolver):

input_corrector: "Todo o mundo chegou a tempo á xuntanza aquel día."

output_corrector: "Todo o mundo chegaron a tempo á xuntanza aquel día."

tag: 1

explanation: "O output_corrector non é adecuado con respecto á concordancia de número porque o corrector engadiu un erro que non existía no input_corrector."





Agora, avalía os seguintes casos:
'''

SELENE_USER_TEMPLATE = r'''You are tasked with evaluating a response based on a given user input and binary scoring rubric and reference answer that serve as the evaluation standard.that serves as the evaluation standard. Provide comprehensive feedback on the response quality strictly adhering to the scoring rubric, followed by a binary tag judgment (0 or 1). Avoid generating any additional opening, closing, or explanations.
Here are some rules of the evaluation: (1) You should prioritize evaluating whether the response satisfies the provided rubric. The basis of your score should depend exactly on the rubric. However, the response does not need to explicitly address points raised in the rubric. Rather, evaluate the response based on the criteria outlined in the rubric. (2) You should refer to the provided reference answer as a guide for evaluating the response.
Your reply must strictly follow exactly this format (no extra text):
input_corrector: "<oración de entrada do corrector>"
output_corrector: "<oración xa avaliada polo corrector>"
tag: <0 ou 1>
explanation: "<explicación breve e precisa en galego>"  Here is the data: Instruction: ``` {user_input} ``` Response: ``` {assistant_response} ``` Score Rubrics: [{rubric_objective}] Tag 0: {rubric_yes_description} Tag 1: {rubric_no_description} Reference answer: {reference_response}'''

RUBRIC_OBJECTIVE = "Concordancia de número: decidir se output_corrector non contén erros de concordancia de número (singular/plural) respecto a determinante, substantivo, adxectivo ou verbo."
RUBRIC_YES = "0/Si: o output_corrector é correcto respecto á concordancia de número (non hai erro)."
RUBRIC_NO = "1/Non: o output_corrector é incorrecto respecto á concordancia de número (segue habendo erro ou introduciuse un novo erro)."
REFERENCE_RESPONSE = ""

def build_prompt(input_text: str, output_text: str) -> str:
    full_instruction = (
        USER_PROMPT
        + "\n\n"
        + f'input_corrector: "{input_text}"\n'
        + f'output_corrector: "{output_text}"\n'
    )
    assistant_response = f'output_corrector: "{output_text}"'
    return SELENE_USER_TEMPLATE.format(
        user_input=full_instruction,
        assistant_response=assistant_response,
        rubric_objective=RUBRIC_OBJECTIVE,
        rubric_yes_description=RUBRIC_YES,
        rubric_no_description=RUBRIC_NO,
        reference_response=REFERENCE_RESPONSE,
    )


In [3]:
# BLOQUE: Post-procesado
# Selene ás veces pode devolver texto extra. Aquí extraemos tag e explanation
# e reconstruímos a saída exactamente co formato que precisamos.

import re

TAG_RE = re.compile(r"\btag\s*:\s*([01])\b", re.IGNORECASE)
EXPL_RE = re.compile(r"\bexplanation\s*:\s*\"?(.*?)\"?\s*$", re.IGNORECASE | re.DOTALL)

def force_strict_output(input_text: str, output_text: str, raw_model_text: str) -> str:
    # 1) intentamos sacar a tag
    m_tag = TAG_RE.search(raw_model_text)
    tag = m_tag.group(1) if m_tag else "0"
    # 2) intentamos sacar a explanation
    m_expl = EXPL_RE.search(raw_model_text.strip())
    explanation = m_expl.group(1).strip() if m_expl else raw_model_text.strip()
    # Limpeza básica para evitar saltos de liña longos na explicación
    explanation = " ".join(explanation.split())
    # 3) reconstruímos exactamente o formato pedido
    return (
        f'input_corrector: "{input_text}"\n'
        f'output_corrector: "{output_text}"\n'
        f'tag: {tag}\n'
        f'explanation: "{explanation}"'
    )


In [4]:
# Cargamos el archivo CSV que se sube a Colab.
# Debe contener las columnas input_corrector y output_corrector.
# Cada fila representa un par input-output del corrector.

file_path = "/content/frases_corrector.csv"
df = pd.read_csv(file_path, encoding='UTF-8', delimiter=';')

In [5]:
# Token HF
HF_TOKEN = os.environ.get("HF_TOKEN", None)

In [6]:
# Selene-1-Mini-Llama-3.1-8B
# LLM-as-a-Judge binario para concordancia de número en galego

# Cargar modelo e tokenizer
MODEL_ID = "AtlaAI/Selene-1-Mini-Llama-3.1-8B"

print("🔄 Cargando tokenizer...")
tokenizer = AutoTokenizer.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN
)

print("🔄 Cargando modelo (pode tardar uns minutos)...")
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    token=HF_TOKEN,
    device_map="auto",
    torch_dtype=torch.bfloat16
)

model.eval()
print("✅ Modelo cargado correctamente")

🔄 Cargando tokenizer...


config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/325 [00:00<?, ?B/s]

...

model.safetensors.index.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

Loading weights:   0%|          | 0/291 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

✅ Modelo cargado correctamente


In [7]:
results = []

In [ ]:
for i, row in df.iterrows():

    input_text = row["input_corrector"]
    output_text = row["output_corrector"]

    prompt = build_prompt(input_text, output_text)

    # Tokenizo el prompt y lo envío al mismo dispositivo que el modelo
    # Selene usa o template de conversa de Llama 3.x cando está dispoñible
    if hasattr(tokenizer, 'apply_chat_template') and getattr(tokenizer, 'chat_template', None):
        messages = [{'role': 'user', 'content': prompt}]
        inputs = tokenizer.apply_chat_template(messages, tokenize=True, add_generation_prompt=True, return_tensors='pt')
    else:
        inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generación determinista
    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=300,
            temperature=0,
            do_sample=False
        )

    # Recorto los tokens correspondientes al prompt para quedarme solo con lo generado
    generated_tokens = outputs[0][inputs["input_ids"].shape[-1]:]

    decoded = tokenizer.decode(generated_tokens, skip_special_tokens=True)

    # Guardo el resultado
    results.append(decoded)

    # Lo imprimo en pantalla
    print("\nResultado do exemplo", i + 1)
    print(decoded)
    print()

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.12/dist-packages/transformers/generation/utils.py:2606: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado do exemplo 1
input_corrector: "As eleccións para escoller o novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio."
output_corrector: "As eleccións para elixir o novo presidente da comunidade non tiveron lugar por mor do mal ambiente no condominio."
tag: 0
explanation: "O output_corrector é correcto respecto á concordancia de número porque mantén a concordancia entre o determinante “eleccións” e o verbo “tiveron”, o que indica que non houbo erro de concordancia de número."



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado do exemplo 2
input_corrector: "O grupo de estudantes do proxecto presentaron os resultados finais onte pola tarde."
output_corrector: "O grupo de estudantes do proxecto presentou os resultados finais onte pola tarde."
tag: 0
explanation: "O output_corrector é correcto respecto á concordancia de número porque corrixiu o erro do input_corrector entre “presentaron” e “grupo”, cambiando o plural por o singular adecuado."



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado do exemplo 3
input_corrector: "As propostas que chegaron desde os concellos máis pequenos foi analizadas polo equipo técnico."
output_corrector: "As propostas que chegaron desde os concellos máis pequenos foron analizadas polo equipo técnico."
tag: 0
explanation: "O output_corrector é correcto respecto á concordancia de número porque solucionou o erro do input_corrector entre “fui” e “propostas”, cambiando “fui” por “foron” para concordar con o plural.”



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.



Resultado do exemplo 4
input_corrector: "Todas as persoas entrevistadas manifestou a súa opinión durante a sesión pública."
output_corrector: "Todas as persoas entrevistadas manifestaron a súa opinión durante a sesión pública."
tag: 0
explanation: "O output_corrector é correcto respecto á concordancia de número porque solucionou o erro do input_corrector entre “manifestou" e “persoas”, cambiando a forma verbal para concordar con o plural."

